Copyright © 2020, Weta Digital, Ltd.

SPDX-License-Identifier: Apache-2.0

# PhysLight Imaging

Here's a very simple idealized "renderer" that calculates the sRGB (linear) pixel values given a $2.5lx$ uniform environment light illuminating a 100% diffuse reflector.

We'll set default camera parameters according to the exposure equation and verify that our output pixel values are exactly 1.

In [102]:
!pip install -q colour-science
!pip install -q matplotlib

import colour
import numpy as np
import math


In [103]:
# couple of utility functions that wrap colour's functionality to make the code clearer down below
def spectral_to_XYZ(sd):
  return colour.sd_to_XYZ(colour.sd_ones(), illuminant=sd, k=1)

def to_photometric(sd):
  # luminous_flux just calculates the integral of the given SpectralDistribution
  # multiplied by the photopic response function times 683 lm/W
  return colour.luminous_flux(sd)


We want to check our working against photometric quantities, to do this we'll want to normalize our light source such that its spectral distribution represents a luminance of $1 nit$. We do this by dividing by:
$$K_m \int_{360nm}^{830nm} S(\lambda) V(\lambda) d\lambda$$

where $V(\lambda)$ is the spectral photopic luminous efficiency function and $K_m = 683lm/W$



In [104]:
# D65 is our light source, normalize it
d65 = colour.ILLUMINANTS_SDS['D65'].copy()
spd_light = d65 / to_photometric(d65)
cmf = colour.STANDARD_OBSERVERS_CMFS['CIE 1931 2 Degree Standard Observer']

# Check that our normalized SPD does indeed equal 1 nit
assert to_photometric(spd_light) == 1.0

Our setup is $2.5lx$ incident on a 100% diffuse reflector. So exitant luminance from the surface, $L_v$ will be $\frac{2.5}{\pi} nit$

In [105]:
L_v = 2.5 / math.pi
# L is radiance scaled to $L_v nit$
L = spd_light * L_v

EV settings from Wikipedia $2.5lx$ $EV0$ example.


https://en.wikipedia.org/wiki/Exposure_value#Relationship_of_EV_to_lighting_conditions

When $EV=0$ (i.e. $2.5lx$ assuming $C=250$), then we should get a "correct" exposure
with these camera settings

In [106]:
t = 1.0
N = 1.0
S = 100.0
C = 250.0
K_m = 683.0


Convert radiance entering the camera system to exposure in $W m^{-2} nm^{-1} s$ (ish - we're actually representing some sort of output signal from the sensor here rather than exposure at the sensor, but it's easier to think of it this way)


In [107]:
imaging_ratio = (math.pi * t * S * K_m) / (C * N * N)
H = L * imaging_ratio

Convert to XYZ then to linear sRGB. We get back to exactly 1 in RGB by dividing by the RGB whitepoint

In [108]:
H_xyz = spectral_to_XYZ(H)

model = colour.models.sRGB_COLOURSPACE

white_xyz = colour.sd_to_XYZ(sd=colour.sd_ones(), illuminant=spd_light) / 100
white_rgb = colour.XYZ_to_RGB(white_xyz, model.whitepoint, model.whitepoint, model.XYZ_to_RGB_matrix)

H_rgb = colour.XYZ_to_RGB(H_xyz, model.whitepoint, model.whitepoint, model.XYZ_to_RGB_matrix) / white_rgb
print('H_rgb', H_rgb)
assert np.array_equal(np.round(H_rgb, 5), [1.0, 1.0, 1.0])

H_rgb [ 0.99999892  0.99999892  0.99999892]
